In [1]:
import pandas as pd
import pandas.io.sql as PBD

In [9]:
def importar_CSV(ruta):
    print("--------------------------------------------")

    try:
        df=pd.read_csv(ruta, delimiter=",", encoding="ISO-8859-1")
        print("Se ha importado el archivo CSV correctamente")

        return(df)

    except PBD.DatabaseError as error:
        print("Error. No se ha podido importar de CSV")
        print(error)

    print("--------------------------------------------")


def exportar_CSV(ruta,df):
    print("--------------------------------------------")

    try:
        df.to_csv(ruta, sep=";", index=False) #index a False para no escribir la columna índice
        print("Se ha exportado el archivo CSV correctamente")

    except PBD.DatabaseError as error:
        print("Error. No se ha podido exportar a CSV")
        print(error)

    print("--------------------------------------------")

def consultar_atributos(df):
    try:
        print("-------------------------")
        print("---consultar_atributos---")
        print("-------------------------")

        print("---head - Primeros valores (5 por defecto, 3 en este caso)---") # Por defecto muestra 5 elementos
        print(df.head(3)) # Mostrar los 3 primeros elementos

        print("---tail - Últimos valores (5 por defecto)---") # Por defecto muestra 5 elementos
        print(df.tail())

        print("---info - Información sobre el archivo CSV (columnas, uso de memoria, tipos...)---")
        print(df.info())

        print("---shape - Número de filas y columnas, respectivamente---")
        print(df.shape)

        print("---size - Número total de casillas (filas por columnas)---")
        print(df.size)

        print("---columns - Columnas---")
        print(df.columns)

        print("---index - Índice de la tabla---")
        print(df.index)

        print("---dtypes - Tipos de datos---")
        print(df.dtypes)

        print("--------------------------------")
        print("---Fin de consultar_atributos---")
        print("--------------------------------")

    except PBD.DatabaseError as error:
        print("Error. Problema en atributos de dataFrame")
        print(error)


In [11]:
!pip install neo4j

In [19]:
from neo4j import GraphDatabase

class HungerGamesApp:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def delete_all_database(self):
        with self.driver.session() as session:
            query = "MATCH (n)-[r]-() DELETE r, n"
            session.run(query)
        
    def create_neo4j_district_nodes(self, values):
        with self.driver.session() as session:
            for val in values:
                if val != 0:
                    session.run(f"CREATE (:District {{Name: 'District {val}', Number: {val}}});\n")
                elif val == 0:
                    session.run(f"CREATE (:District {{Name: 'The Capitol', Number: {val}}});\n")
                else:
                    session.run(f"CREATE (:District {{Name: '???', Number: -1}});\n")

    def create_neo4j_game_year_nodes(self, games):
        all_years = set()
        for val in games:
            if pd.isna(val):
                continue
            parts = [a.strip() for a in str(val).split(",")]
            all_years.update(parts)

        with self.driver.session() as session:
            for year in sorted(all_years):
                session.run(f"CREATE (:Game_Year {{Year: {year}}});\n")

    def create_neo4j_book_nodes(self, books):
        all_books = set()
        for val in books:
            if pd.isna(val):
                continue
            parts = [a.strip() for a in str(val).split(",")]
            all_books.update(parts)

        with self.driver.session() as session:
            for book in sorted(all_books):
                safe_name = book.replace("'", "`")
                order = -1

                match safe_name:
                    case "The Hunger Games":
                        order = 1
                    case "Catching Fire":
                        order = 2
                    case "Mockingjay":
                        order = 3
                    case "The Ballad of Songbirds and Snakes":
                        order = 4
                    case "Sunrise on the Reaping":
                        order = 5
                    case "Mentioned":
                        order = 6

                if safe_name not in ["Trilogy"]:
                    session.run(f"CREATE (:Book {{Title: '{safe_name}', Order: '{order}'}});\n")

    def create_neo4j_alliance_nodes(self, alliances):
        all_alliances = set()
        for val in alliances:
            if pd.isna(val):
                continue
            parts = [a.strip() for a in str(val).split(",")]
            all_alliances.update(parts)

        with self.driver.session() as session:
            for alliance in sorted(all_alliances):
                safe_name = alliance.replace("'", "`")
                if safe_name not in ["Katniss", "Haymitch"]:
                    session.run(f"CREATE (:Alliance {{Name: '{safe_name}'}});\n")

    def create_neo4j_character_nodes(self, df, filename="cypher_files/create_characters.cypher"):
        allowed_columns = ["ID", "Name", "Gender", "Profession"]
        with self.driver.session() as session:
            for _, row in df.iterrows():
                props_list = []
                for col in allowed_columns:
                    if col in df.columns:
                        value = row[col]
                        if pd.isna(value) and col == "Profession":
                            value = "None"
                        if not pd.isna(value):
                            safe_value = str(value).replace("'", "`")
                            props_list.append(f"{col}: '{safe_value}'")
                props = ", ".join(props_list)
                session.run(f"CREATE (:Character {{{props}}});\n")
    # -------------------------------------------------------------------------------------

    def create_character_district_links(self, df):
        with self.driver.session() as session:
            for _, row in df.iterrows():
                id = str(row["ID"])
                district = row["District"]

                if pd.isna(id) or pd.isna(district):
                    continue

                session.run(
                    f"MATCH (c:Character {{ID: '{id}'}}), (d:District {{Number: {int(district)}}})\n"
                    f"CREATE (c)-[:FROM_DISTRICT]->(d);\n"
                )

    def create_character_game_links(self, df):
        with self.driver.session() as session:
            for _, row in df.iterrows():
                char_id = str(row["ID"])
                games = row.get("Game_Year", None)
                winner = str(row.get("Winner", "No")).strip().lower()

                if pd.isna(char_id) or pd.isna(games):
                    continue

                winner_flag = "true" if winner == "yes" else "false"

                years = [g.strip() for g in str(games).split(",")]

                for year in years:
                    if year.isdigit():
                        year_int = int(year)
                        if year_int == 75:
                            session.run(
                                f"MATCH (c:Character {{ID: '{char_id}'}}), (g:Game_Year {{Year: {year_int}}})\n"
                                f"CREATE (c)-[:PARTICIPATED_IN {{victor: false}}]->(g);\n"
                            )
                        else:
                            session.run(
                                f"MATCH (c:Character {{ID: '{char_id}'}}), (g:Game_Year {{Year: {year_int}}})\n"
                                f"CREATE (c)-[:PARTICIPATED_IN {{victor: {winner_flag}}}]->(g);\n"
                            )

    def create_character_book_links(self, df):
        trilogy_books = ["The Hunger Games", "Catching Fire", "Mockingjay"]

        with self.driver.session() as session:
            for _, row in df.iterrows():
                char_id = str(row["ID"])
                books = row.get("Appearance", None)

                if pd.isna(char_id) or pd.isna(books):
                    continue

                book_list = [b.strip() for b in str(books).split(",")]

                for book in book_list:
                    if book == "Trilogy":
                        expanded_books = trilogy_books
                    elif book != "None":
                        expanded_books = [book]
                    else:
                        continue

                    for title in expanded_books:
                        safe_book = title.replace("'", "`")
                        session.run(
                            f"MATCH (c:Character {{ID: '{char_id}'}}), (b:Book {{Title: '{safe_book}'}})\n"
                            f"CREATE (c)-[:APPEARS_IN]->(b);\n"
                        )

    def create_character_alliance_links(self, df):
        with self.driver.session() as session:
            for _, row in df.iterrows():
                char_id = str(row["ID"])
                alliances = row.get("Alliance", None)

                if pd.isna(char_id) or pd.isna(alliances):
                    continue

                alliance_list = [a.strip() for a in str(alliances).split(",")]

                for alliance in alliance_list:
                    if alliance == "Katniss":
                        session.run(
                            f"MATCH (c:Character {{ID: '{char_id}'}}), (k:Character {{Name: 'Katniss Everdeen'}})\n"
                            f"CREATE (c)-[:ALLY_OF]->(k);\n"
                        )
                    elif alliance == "Haymitch":
                        session.run(
                            f"MATCH (c:Character {{ID: '{char_id}'}}), (h:Character {{Name: 'Haymitch Abernathy'}})\n"
                            f"CREATE (c)-[:ALLY_OF]->(h);\n"
                        )
                    else:
                        safe_alliance = alliance.replace("'", "`")
                        session.run(
                            f"MATCH (c:Character {{ID: '{char_id}'}}), (a:Alliance {{Name: '{safe_alliance}'}})\n"
                            f"CREATE (c)-[:BELONGS_TO]->(a);\n"
                        )

    def create_neo4j_mentor_links(self, df):
        with self.driver.session() as session:
            for _, row in df.iterrows():
                char_id = str(row["ID"])
                mentors = row.get("Mentor", None)

                if pd.isna(char_id) or pd.isna(mentors):
                    continue

                mentor_list = [a.strip() for a in str(mentors).split(",")]
                for mentor in mentor_list:
                    safe_mentor = mentor.replace("'", "`")
                    session.run(
                        f"MATCH (c:Character {{ID: '{char_id}'}}), (m:Character {{Name: '{safe_mentor}'}})\n"
                        f"CREATE (m)-[:MENTORS]->(c);\n"
                    )

    # -------------------------------------------------------------------------------------


In [13]:
ruta_csv='HungerGames_Characters_Dataset_ALL.csv'

In [20]:

print("------------------------")
print("---Programa principal---")
print("------------------------")

hgDF = importar_CSV(ruta_csv)
hgDF = hgDF.drop('Columna 1', axis=1)

hgDF['ID'] = hgDF.index

uri = "bolt://localhost:7687"
user = "mdad"
password = "123456789"

app = HungerGamesApp(uri, user, password)
try:
    # Create the cypher files for creating nodes
    districts = hgDF['District'].unique()
    print("\nUnique values in 'District' column:", districts)
    app.create_neo4j_district_nodes(districts)

    games = hgDF['Game_Year'].unique()
    print("\nUnique values in 'Game_Year' column:", games)
    app.create_neo4j_game_year_nodes(games)

    books = hgDF['Appearance'].unique()
    print("\nUnique values in 'Appearance' column:", books)
    app.create_neo4j_book_nodes(books)

    alliances = hgDF['Alliance'].unique()
    print("\nUnique values in 'Alliance' column:", alliances)
    app.create_neo4j_alliance_nodes(alliances)

    app.create_neo4j_character_nodes(hgDF)

    # Create the cypher files for linking
    app.create_character_district_links(hgDF)
    app.create_character_game_links(hgDF)
    app.create_character_book_links(hgDF)
    app.create_character_alliance_links(hgDF)
    app.create_neo4j_mentor_links(hgDF)

    consultar_atributos(hgDF)
finally:
    app.close()

print("----------------------")
print("---Fin del programa---")
print("----------------------")


------------------------
---Programa principal---
------------------------
--------------------------------------------
Se ha importado el archivo CSV correctamente

Unique values in 'District' column: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13]

Unique values in 'Game_Year' column: [nan '10' '67' '50' '64, 75' '74' '63, 75' '46' '75' '62, 75' '73'
 '34, 75' '49, 75' '70' '75, 65' '11, 75' '38' '66,75' '71, 75' '15, 75'
 '68' '45, 75' '75, 74' '74, 75']

Unique values in 'Appearance' column: ['The Ballad of Songbirds and Snakes' 'Trilogy'
 'Trilogy, Sunrise on the Reaping' 'Sunrise on the Reaping'
 'The Hunger Games, Catching Fire, Mentioned'
 'Sunrise on the Reaping, Trilogy, The Ballad of Songbirds and Snakes'
 'The Ballad of Songbirds and Snakes, Mentioned' 'Mockingjay'
 'The Hunger Games, Mentioned' 'Catching Fire'
 'Mockingjay, The Ballad of Songbirds and Snakes' 'Mentioned'
 'The Hunger Games' 'Catching Fire, Mockingjay'
 'Sunrise on the Reaping, Catching Fire'
 'Catching Fire, S